# Induced gap and magnetic field

In [ ]:
# 1. Standard library imports
import asyncio
from copy import copy
from functools import partial
from operator import itemgetter

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np

# 3. Internal imports
import funcs
import common
from common import loss

# hv.notebook_extension('matplotlib')
print(kwant.__version__)

import adaptive
adaptive.notebook_extension()

# Connect to ipyparallel

In [ ]:
import hpc05
hpc05.kill_remote_ipcluster()
client, dview, lview = hpc05.start_remote_and_connect(200, profile='pbs2',
                                                      folder='~/Work/induced_gap_B_field/')
# client, dview, lview = hpc05.connect_ipcluster(100, profile='pbs2', folder='~/Work/induced_gap_B_field/')

# Usage

In [ ]:
syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=200, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=60, g=50, orbital=True, mu_sc=100,
              mu=7, mu_lead=7, c_tunnel=3/4, V=lambda x, y, z: 0, **funcs.constants.__dict__)

syst_pars['L_barrier'] = 100
params['V_barrier_mu'] = syst_pars['L_barrier']/2
params['V_barrier_sigma'] = syst_pars['L_barrier']/10
params['V_barrier_height'] = 30
params['V_barrier'] = funcs.V_barrier

syst = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst);

In [ ]:
lead = syst.leads[0]
funcs.plot_wfs_in_cross_section(lead, params, 0)

In [ ]:
%%opts Image [colorbar=True]

x = 20
xy = funcs.get_cross_section(syst, x, 0)

V = lambda x, y, z: -2 * z / syst_pars['r1']

potential = np.array([V(*s.pos) for s in syst.sites if s.pos[0]==x])

hv.Image(np.rot90(kwant.plotter.mask_interpolate(xy, potential, oversampling=1)[0]))

In [ ]:
# %%opts Path [aspect='square']
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=20, orbital=True, V=lambda x, y, z: 0, Delta=60,
              **funcs.constants.__dict__)

lead_pars = dict(a=10, angle=0,
                 coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_shell=True, A_correction=True)

lead = funcs.make_lead(**lead_pars).finalized()
params['mu_lead'] = params['mu_sc'] = 10
params['B_y'] = 0
ks = np.linspace(-1, 1)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

p1[:, -25:25] + p1[:, -2:2]

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print(f'The bandgap is {gap} meV')

In [ ]:
params = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=15, mu_lead=15, c_tunnel=3/4, V=lambda x,y,z:0, V_barrier=50, **funcs.constants.__dict__)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params)

# Tuning the gap

In [ ]:
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=20, orbital=True, V='lambda x, y, z: 10 * z / 35', mu_sc=100,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, coverage_angle=180, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True)

def lowest_energy(x, syst_pars, params):
    import funcs
    import numpy as np
    lead = funcs.make_lead(**syst_pars).finalized()
    params['mu_lead'], params['Delta'] = x
    # Combine the fixed parameters `params` and the changing
    # parameters `val` to one dict and evaluate the string
    # lambda functions.
    params = funcs.parse_params(params)

    # Create the Hamiltonian `ham` at k=0.
    h0 = lead.cell_hamiltonian(params=params)
    t0 = lead.inter_cell_hopping(params=params)
    ham = h0 + t0 + t0.conj().T

    # Find the energies.
    ev = np.linalg.eigvalsh(ham)
    
    # Return a combined dictionary with the results and input.
    return np.abs(ev).min()

def find_crossings(f, g):
    return np.argwhere(np.diff(np.sign(f - g)) != 0).reshape(-1)

def get_Delta(learner, Delta_ind):
    plot = learner.plot(n_x=1000, n_y=1000).redim(x='mu', y='Delta', z='E_gap')
    line = plot.reduce(['mu'], np.min)
    idx = find_crossings(line.data['E_gap'], Delta_ind)[-1]
    Delta = line.data['Delta'][idx]
    return Delta

In [ ]:
learner = adaptive.Learner2D(partial(lowest_energy, syst_pars=syst_pars, params=params),
                             [(5, 20), (0, 200)], loss)

runner = adaptive.Runner(learner, client, goal=lambda l: l.loss() < 0.001)

In [ ]:
runner.task.done()

In [ ]:
%%opts Image [aspect='square']
plot = learner.plot().redim(x='mu', y='Delta', z='E_gap')
plot

In [ ]:
%%opts Scatter (size=10, color='r')
Delta_ind = 0.4

line = plot.reduce(['mu'], np.min)
line * hv.HLine(0.4) * hv.Scatter((get_Delta(learner, Delta_ind), Delta_ind))

In [ ]:
Deltas = np.arange(*learner.bounds[1])
hm = hv.HoloMap({Delta: plot.sample(Delta=Delta)[:, 0.3:0.7] for Delta in Deltas})
hm * hv.HLine(0.4)

#### Run it for different potentials

In [ ]:
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=40, g=50,
              alpha=20, orbital=True, mu_sc=100,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=90, coverage_angle=180, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True)

learners = []
gradients = np.arange(-10, 8, 2)
for grad in gradients:
    params['V'] = f'lambda x, y, z: {grad} * z / 35'
    f = partial(lowest_energy, syst_pars=syst_pars, params=copy(params))
    learner = adaptive.Learner2D(f, [(5, 20), (0, 250)], loss)
    learner.cdims = grad
    learners.append(learner)
    
learner = adaptive.BalancingLearner(learners)

In [ ]:
runner = adaptive.Runner(learner, client, goal=lambda l: l.loss() < 0.0001)

In [ ]:
runner.task.print_stack(), learner.loss(), sum(l.n for l in learner.learners)

In [ ]:
hv.HoloMap(({l.cdims: l.plot().redim(x='mu', y='Delta', z='E_gap') for l in learner.learners}))

In [ ]:
Delta_ind = 0.4
plots = {}
gaps = {}
for l in learner.learners:
    plot = l.plot(n_x=1000, n_y=1000).redim(x='mu', y='Delta', z='E_gap')
    line = plot.reduce(['mu'], np.min)
    try:
        Delta = get_Delta(l, Delta_ind)
    except IndexError:
        Delta = np.nan
    plots[l.cdims] = line * hv.HLine(0.4) * hv.Scatter((Delta, Delta_ind))
    gaps[l.cdims] = Delta

In [ ]:
print(gaps)  # Use these in the "Sweep electric field gradient" simulation below.

In [ ]:
%%opts Scatter (size=5 c='r')
hv.HoloMap(plots, kdims=['gradient'])

# conductance $V_{bias}$ vs $B_x$

In [ ]:
def conductance(x, val, syst_pars, params):
    import funcs

    for k in ['angle']:
        syst_pars[k] = val[k]

    params = funcs.parse_params(dict(**params, **val))
    
    params['mu_lead'] = params['mu']
    params['B_x'], params['B_y'], params['B_z'] = (0, 0, 0)
    val['V_bias'], params['B_{}'.format(val['direction'])] = x

    syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val['V_bias'])

### Sweep electric field gradient

In [ ]:
gaps = {-10: {0: 201.375, 90: 125.875},
        -8: {0: 178.375, 90: 116.625},
        -6: {0: 152.875, 90: 110.125},
        -4: {0: 129.625, 90: 103.125},
        -2: {0: 107.125, 90: 97.375},
         0: {0: 85.875, 90: 94.875},
         2: {0: 65.625, 90: 93.625},
         4: {0: 46.875, 90: 94.625},
#          6: {0: 30.125, 90: 97.375},
       }

params = dict(c_tunnel=3/4, V_barrier=40, mu_sc=100, mu=15,
             **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False, 
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           direction=list('xyz'),
                           gradient=list(gaps.keys()),
                           angle=[0, 90])

learners = []
for val in vals:
    params['V'] = f'lambda x, y, z: {val["gradient"]} * z / {syst_pars["r1"]}'
    params['Delta'] = gaps[val['gradient']][val['angle']]
    f = partial(conductance, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive.Learner2D(f, [(-1, 1), (0, 2)], loss)
    learner.cdims = val
    learner.stack_size = 40
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)
folder = 'data/gradient-sweep-angle-0-90'
# funcs.load_BalancingLearner_data(learners, folder)

### Sweep chemical potential

In [ ]:
syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

params = dict(Delta=85.875, c_tunnel=3/4, V='lambda x, y, z: 0', mu_sc=100,
             **funcs.constants.__dict__)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           direction=list('xyz'),
                           mu=list(range(10, 21)))

learners = []
for val in vals:
    f = partial(conductance, val=val, params=params, syst_pars=syst_pars)
    learner = adaptive.Learner2D(f, [(-1, 1), (0, 2)], loss)
    learner.cdims = val
    learner.stack_size = 20
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)
folder = 'data/mu-sweep'
# funcs.load_BalancingLearner_data(learners, folder)

### Run

In [ ]:
runner = adaptive.Runner(learner, executor=client, 
                         shutdown_executor=False, log=False)

ioloop = asyncio.get_event_loop()
# backup = ioloop.create_task(common.periodic_data_saver(runner, interval=7200))

In [ ]:
funcs.save_BalancingLearner_data(learners, folder)

In [ ]:
# runner.task.cancel()
# backup.cancel()
runner.task.print_stack()

In [ ]:
sum(l.n for l in learners)

In [ ]:
idx = np.argmin([l.n for l in learner.learners])
learner._points[idx][-1][0] = np.inf
idx, learners[idx].n

In [ ]:
%%output size=150
learner.learners[idx].plot(tri_alpha=0.3)

In [ ]:
%%opts Image {+framewise} [colorbar=True]
%%output size=200

plots = {tuple(val.values()): l.plot(201, 201, 0) for (l, val) in zip(learner.learners, vals)}
hm = hv.HoloMap(plots, kdims=list(vals[0].keys()))
hm

In [ ]:
hv.util.Dynamic(hm)